In [8]:
import PySimpleGUI as sg
import sqlite3
import uuid
import random

conn = sqlite3.connect('Project.db')
cursor = conn.cursor()


create_table_sql = '''
CREATE TABLE IF NOT EXISTS Discounts (
    discount_code TEXT PRIMARY KEY,
    start_date DATE,
    end_date DATE,
    discount_percentage INTEGER
);
'''

#Run this once to create the Orders table
create_orders_table_sql = '''
CREATE TABLE IF NOT EXISTS Orders (
    order_id TEXT PRIMARY KEY,
    customer_id TEXT,
    receiver_name TEXT,
    receiver_address TEXT,
    receiver_contact TEXT,
    total_price REAL,
    order_date DATE DEFAULT CURRENT_DATE,
    delivery_status TEXT DEFAULT "incomplete",
    expected_delivery_date DATE DEFAULT NULL,
    gift_note TEXT,
    deliverer_id TEXT
);
'''
cursor.execute(create_orders_table_sql)
conn.commit()



create_orders_table_sql2 = '''
CREATE TABLE IF NOT EXISTS DelivererAssignments (
    assignment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    deliverer_id TEXT,
    flower_id TEXT,
    available_for_assignment BOOLEAN DEFAULT 1,
    FOREIGN KEY(deliverer_id) REFERENCES Users(ID),
    FOREIGN KEY(flower_id) REFERENCES Floral_Arrangement(flower_ID)
);
'''
cursor.execute(create_orders_table_sql2)
conn.commit()

# Execute the SQL command
cursor.execute(create_table_sql)

create_orders_table_sql1 = '''
CREATE TABLE IF NOT EXISTS OrderDetails (
    detail_id INTEGER PRIMARY KEY,
    order_id TEXT,
    flower_id TEXT,
    quantity INTEGER,
    FOREIGN KEY(order_id) REFERENCES Orders(order_id),
    FOREIGN KEY(flower_id) REFERENCES Floral_Arrangement(flower_ID)
);

'''
cursor.execute(create_orders_table_sql1)
conn.commit()



login_layout = [
    [sg.Text("Please enter your credentials")],
    [sg.Text("ID", size =(15, 1)), sg.InputText(key='ID')],
    [sg.Text("Password", size =(15, 1)), sg.InputText(key='Password', password_char='*')],
    [sg.Button("Login"), sg.Button("Cancel")]
]

def generate_order_id():
    return str(random.randint(1,100))

def create_admin_window():
    layout = [
        [sg.Text("Admin Panel")],
        [sg.Button("Add Flower Arrangement"), sg.Button("Manage Floral Arrangements"), sg.Button("Define Discounts"), sg.Button("View Discounts"), sg.Button("Assign Orders to Deliverers")],
        [sg.Button("Logout")]
    ]
    return sg.Window("Admin", layout, finalize=True)



def show_flower_details(flower_id):
    query = "SELECT * FROM Floral_Arrangement WHERE flower_ID=?"
    cursor.execute(query, (flower_id,))
    flower = cursor.fetchone()
    details = f"ID: {flower[0]}\nSize: {flower[4]}\nType: {flower[3]}\nQuantity: {flower[2]}\nPrice: {flower[1]}\nName: {flower[5]}\nDesign: {flower[6]}"
    sg.popup(f"Details for {flower[5]}", details, title="Flower Details")

def create_customer_window():
    cursor.execute("SELECT flower_ID, f_name, f_price FROM Floral_Arrangement")
    flowers = cursor.fetchall()
    layout = [
        [sg.Text("Select a Flower Arrangement:")],
        [sg.Listbox(values=[f"{flower[0]} - {flower[1]} - ${flower[2]}" for flower in flowers], size=(40, 10), key='FlowerList', select_mode=sg.LISTBOX_SELECT_MODE_MULTIPLE)],
        [sg.Button("View Details"), sg.Button("Add to Cart"), sg.Button("View Cart"), sg.Button("View Order History"), sg.Button("Exit")]
    ]
    return sg.Window("Customer View", layout)

def create_flower_window(mode):
    cursor.execute("SELECT flower_ID FROM Floral_Arrangement")
    flowers = cursor.fetchall()
    flower_layout = [
        [sg.Text("Select Flower ID"), sg.Listbox(values=[f[0] for f in flowers], size=(20, 5), key='FlowerList')],
        [sg.Button("Select"), sg.Button("Back")]
    ]
    
    layout = [
        [sg.Text(f"{mode} Flower Arrangement")],
        flower_layout if mode != "Add" else []
    ]

    if mode == "Add" or mode == "Edit":
        layout.extend([
            [sg.Text("Flower ID", size =(15, 1)), sg.InputText(key='flower_ID')],
            [sg.Text("Size", size =(15, 1)), sg.InputText(key='f_size')],
            [sg.Text("Type", size =(15, 1)), sg.InputText(key='f_type')],
            [sg.Text("Quantity", size =(15, 1)), sg.InputText(key='quantity_in_stock')],
            [sg.Text("Price", size =(15, 1)), sg.InputText(key='f_price')],
            [sg.Text("Name", size =(15, 1)), sg.InputText(key='f_name')],
            [sg.Text("Design", size =(15, 1)), sg.InputText(key='f_design')],
            [sg.Button("Submit"), sg.Button("Cancel")]
        ])
    return sg.Window(f"{mode} Flower", layout)

def create_manage_flowers_window():
    cursor.execute("SELECT flower_ID FROM Floral_Arrangement")
    flowers = cursor.fetchall()
    layout = [
        [sg.Text("Manage Flower Arrangements")],
        [sg.Listbox(values=[f[0] for f in flowers], size=(20, 10), key='FlowerList')],
        [sg.Button("View"), sg.Button("Edit"), sg.Button("Delete"), sg.Button("Back")]
    ]
    return sg.Window("Manage Flowers", layout)

def create_cart_window(cart_items):
    cart_layout = [
        [sg.Text("Your Cart")],
        [sg.Listbox(values=[f"{item[0]} - {item[1]} - Qty: {item[3]}" for item in cart_items], size=(45, 10), key='CartList')],
        [sg.Button("Remove Selected"), sg.Button("Checkout"), sg.Button("Continue Shopping")]
    ]
    return sg.Window("Shopping Cart", cart_layout)

def add_to_cart(selections, cart_items):
    for selection in selections:
        # Extract necessary information (ensure this matches how you display in Listbox)
        item_details = selection.split(' - ')  # Adjust according to your actual split character
        flower_id = item_details[0]
        flower_name = item_details[1]
        price = item_details[2].strip('$')  # Remove $ if price displayed with it
        found = False
        for item in cart_items:
            if item[0] == flower_id:  # Check if the item already exists in the cart
                item_index = cart_items.index(item)
                # Update the quantity
                cart_items[item_index] = (item[0], item[1], item[2], item[3] + 1)
                found = True
                break
        if not found:
            # Append new item to the cart
            cart_items.append((flower_id, flower_name, price, 1))


def remove_from_cart(selected_item, cart_items):
    flower_id = selected_item.split(' - ')[0]
    for item in cart_items:
        if item[0] == flower_id:
            if item[3] > 1:
                item_index = cart_items.index(item)
                cart_items[item_index] = (item[0], item[1], item[2] - 1)
            else:
                cart_items.remove(item)
            break

def manage_flower_details(flower_id, mode):
    query = "SELECT * FROM Floral_Arrangement WHERE flower_ID=?"
    cursor.execute(query, (flower_id,))
    flower = cursor.fetchone()
    if mode == "View":
        details = f"ID: {flower[0]}\nSize: {flower[4]}\nType: {flower[3]}\nQuantity: {flower[2]}\nPrice: {flower[1]}\nName: {flower[5]}\nDesign: {flower[6]}"
        sg.popup(f"Flower Details for ID {flower_id}", details)
    elif mode == "Edit":
        layout = [
            [sg.Text("Edit Flower Arrangement")],
            [sg.Text("Flower ID", size =(15, 1)), sg.InputText(flower[0], disabled=True, key='flower_ID')],
            [sg.Text("Size", size =(15, 1)), sg.InputText(flower[4], key='f_size')],
            [sg.Text("Type", size =(15, 1)), sg.InputText(flower[3], key='f_type')],
            [sg.Text("Quantity", size =(15, 1)), sg.InputText(flower[2], key='quantity_in_stock')],
            [sg.Text("Price", size =(15, 1)), sg.InputText(flower[1], key='f_price')],
            [sg.Text("Name", size =(15, 1)), sg.InputText(flower[5], key='f_name')],
            [sg.Text("Design", size =(15, 1)), sg.InputText(flower[6], key='f_design')],
            [sg.Button("Update"), sg.Button("Cancel")]
        ]
        edit_window = sg.Window("Edit Flower", layout)
        while True:
            event, values = edit_window.read()
            if event == "Update":
                if any(not values[field].strip() for field in ['f_size', 'f_type', 'quantity_in_stock', 'f_price', 'f_name', 'f_design']):
                    sg.popup_error('All fields are required. Please fill out all fields.')
                elif not values['flower_ID'].isnumeric():
                    sg.popup('flower ıd should be numeric')
                elif not values['quantity_in_stock'].isnumeric():
                    sg.popup('fquantity should be numeric') 
                elif not values['f_price'].isnumeric():
                    sg.popup('fprice should be numeric')
                else:
                    update_query = '''
                    UPDATE Floral_Arrangement SET f_size=?, f_type=?, quantity_in_stock=?, f_price=?, f_name=?, f_design=?
                    WHERE flower_ID=?
                    '''
                    data = (values['f_size'], values['f_type'], values['quantity_in_stock'], values['f_price'], values['f_name'], values['f_design'], flower[0])
                    cursor.execute(update_query, data)
                    conn.commit()
                    sg.popup('Flower arrangement updated successfully!')
                    edit_window.close()
                    break
            elif event in (sg.WIN_CLOSED, 'Cancel'):
                edit_window.close()
                break

def delete_flower(flower_id):
    if sg.popup_yes_no(f"Are you sure you want to delete Flower ID {flower_id}?") == "Yes":
        cursor.execute("DELETE FROM Floral_Arrangement WHERE flower_ID=?", (flower_id,))
        conn.commit()
        sg.popup("Flower arrangement deleted successfully.")

def create_discount_window():
    layout = [
        [sg.Text("Discount Code:", size=(15, 1)), sg.InputText(key='Discount_Code')],
        [sg.Text("Start Date:", size=(15, 1)), sg.InputText(key='Start_Date'), sg.CalendarButton("Select Start Date", target='Start_Date', format='%Y-%m-%d')],
        [sg.Text("End Date:", size=(15, 1)), sg.InputText(key='End_Date'), sg.CalendarButton("Select End Date", target='End_Date', format='%Y-%m-%d')],
        [sg.Text("Discount Percentage:", size=(15, 1)), sg.InputText(key='Discount_Percentage')],
        [sg.Button("Submit"), sg.Button("Cancel")]
    ]
    return sg.Window("Add Discount", layout, modal=True)


from datetime import datetime

def handle_discount_submission(window):
    while True:
        event, values = window.read()
        if event == 'Submit':
            discount_code = values['Discount_Code']
            start_date = values['Start_Date']
            end_date = values['End_Date']
            discount_percentage = values['Discount_Percentage']

            # Convert string dates to datetime objects for comparison
            current_date = datetime.now().date()
            try:
                start_datetime = datetime.strptime(start_date, '%Y-%m-%d').date()
                end_datetime = datetime.strptime(end_date, '%Y-%m-%d').date()
            except ValueError:
                sg.popup_error("Invalid date format. Please select a date using the date picker.")
                continue

            # Check if the start date is before the current date
            if start_datetime < current_date:
                sg.popup_error("Start date cannot be before the current date.")
                continue

            # Check if the end date is in the past
            if end_datetime < current_date:
                sg.popup_error("End date cannot be in the past.")
                continue

            if not discount_percentage.isdigit():
                sg.popup_error('Discount percentage must be an integer.')
                continue

            try:
                cursor.execute("INSERT INTO Discounts (discount_code, start_date, end_date, discount_percentage) VALUES (?, ?, ?, ?)",
                               (discount_code, start_date, end_date, int(discount_percentage)))
                conn.commit()
                sg.popup("Discount added successfully!")
                break
            except sqlite3.IntegrityError:
                sg.popup_error("Error: Discount code already exists.")
        elif event in (sg.WIN_CLOSED, 'Cancel'):
            break
    window.close()


        
def fetch_discounts():
    cursor.execute("SELECT discount_code, start_date, end_date, discount_percentage FROM Discounts")
    return cursor.fetchall()

def create_view_discounts_window():
    discounts = fetch_discounts()
    discount_list = [list(discount) for discount in discounts]
    layout = [
        [sg.Text("Existing Discounts")],
        [sg.Table(
            values=discount_list,
            headings=['Code', 'Start Date', 'End Date', 'Discount %'],
            display_row_numbers=True,
            auto_size_columns=True,
            num_rows=min(10, len(discount_list)),
            key='DiscountTable',
            enable_events=True,
            select_mode=sg.TABLE_SELECT_MODE_BROWSE
        )],
        [sg.Button("Delete Discount"), sg.Button("Close")]
    ]
    discount_window = sg.Window("View Discounts", layout, modal=True)

    while True:
        event, values = discount_window.read()
        if event == "Delete Discount":
            if values['DiscountTable']:  # Ensure a row is selected
                selected_index = values['DiscountTable'][0]  # Get index of selected row
                selected_discount_code = discount_list[selected_index][0]  # Assuming the code is in the first column
                delete_discount(selected_discount_code)  # Call the delete function
                discount_window.close()
                return True  # You can return a flag to indicate a deletion has occurred
            else:
                sg.popup_error("Please select a discount to delete.")
        elif event in ("Close", sg.WIN_CLOSED):
            discount_window.close()
            return False

def delete_discount(discount_code):
    try:
        cursor.execute("DELETE FROM Discounts WHERE discount_code=?", (discount_code,))
        conn.commit()
        sg.popup("Discount code deleted successfully.")
    except Exception as e:
        sg.popup_error(f"An error occurred while trying to delete the discount code: {str(e)}")

def create_gift_note_window():
    layout = [
        [sg.Text("Please enter your gift note:")],
        [sg.Multiline(size=(30, 5), key='GiftNote')],
        [sg.Button("Submit Note"), sg.Button("Cancel")]
    ]
    return sg.Window("Gift Note", layout, modal=True)

def display_discounts_and_select():
    discounts = fetch_discounts()
    discount_list = [list(discount) for discount in discounts]
    layout = [
        [sg.Text("Available Discounts")],
        [sg.Table(
            values=discount_list,
            headings=['Code', 'Start Date', 'End Date', 'Discount %'],
            display_row_numbers=True,
            auto_size_columns=True,
            num_rows=min(10, len(discount_list)),
            key='DiscountTable',
            enable_events=True,
            select_mode=sg.TABLE_SELECT_MODE_BROWSE
        )],
        [sg.Button("Select Discount"), sg.Button("No Discount"), sg.Button("Cancel")]
    ]
    discount_window = sg.Window("View Discounts", layout, modal=True)

    selected_discount = None
    while True:
        event, values = discount_window.read()
        if event == "Select Discount" and values['DiscountTable']:
            selected_discount = discounts[values['DiscountTable'][0]]
            discount_window.close()
            return selected_discount
        elif event in ("No Discount", "Cancel", sg.WIN_CLOSED):
            discount_window.close()
            return None
        
def delete_used_discount(discount_code):
    try:
        cursor.execute("DELETE FROM Discounts WHERE discount_code=?", (discount_code,))
        conn.commit()
        sg.popup("Discount applied")
    except Exception as e:
        sg.popup_error(f"An error occurred while trying to delete the discount code: {str(e)}")
        
def create_receiver_info_window():
    layout = [
        [sg.Text("Receiver's Name:"), sg.InputText(key='ReceiverName')],
        [sg.Text("Receiver's Address:"), sg.Multiline(key='ReceiverAddress')],
        [sg.Text("Receiver's Contact Number:"), sg.InputText(key='ReceiverContact')],
        [sg.Button("Submit"), sg.Button("Cancel")]
    ]
    return sg.Window("Enter Receiver Information", layout, modal=True)

def fetch_order_history(customer_id):
    cursor.execute("""
        SELECT o.order_id, o.receiver_name, o.total_price, o.order_date, o.delivery_status, o.expected_delivery_date, o.gift_note,
               GROUP_CONCAT(d.flower_id, ', ') AS flower_ids
        FROM Orders o
        LEFT JOIN OrderDetails d ON o.order_id = d.order_id
        WHERE o.customer_id=?
        GROUP BY o.order_id
        ORDER BY o.order_date DESC
    """, (customer_id,))
    return cursor.fetchall()





def create_customer_order_history_window(customer_id):
    orders = fetch_order_history(customer_id)
    layout = [
        [sg.Text("Your Order History")],
        [sg.Listbox(values=[
            f"Order ID: {order[0]} - Receiver: {order[1]} - Total: ${order[2]:.2f} - Date: {order[3]} - Status: {order[4]} - Delivery Date: {order[5] if order[5] else 'N/A'} - Gift Note: {order[6]} - Flower IDs: {order[7]}"
            for order in orders], size=(100, 10), key='OrderHistory')],
        [sg.Button("Close")]
    ]
    order_history_window = sg.Window("Order History", layout)

    while True:
        event, values = order_history_window.read()
        if event in ("Close", sg.WIN_CLOSED):
            order_history_window.close()
            break






def create_deliverer_window(deliverer_id):
    # Fetch available arrangements to choose
    cursor.execute("SELECT flower_ID, f_name FROM Floral_Arrangement")
    flowers = cursor.fetchall()

    layout = [
        [sg.Text("Select Arrangements You Can Prepare:")],
        [sg.Listbox(values=[f"{flower[0]} - {flower[1]}" for flower in flowers], size=(30, 10), key='FlowerList', select_mode=sg.LISTBOX_SELECT_MODE_MULTIPLE)],
        [sg.Button("Add Selected"), sg.Button("View Assigned Orders"), sg.Button("Exit")],
        [sg.Text("Current Assignments:")],
        [sg.Listbox(values=[], size=(30, 10), key='CurrentAssignments')],
        [sg.Button("Delete Selected")]
    ]
    # Window is finalized upon creation
    window = sg.Window("Deliverer View", layout, finalize=True)

    # Function to refresh current assignments
    def refresh_current_assignments():
        cursor.execute("SELECT d.flower_id, f.f_name FROM DelivererAssignments d JOIN Floral_Arrangement f ON d.flower_id = f.flower_ID WHERE d.deliverer_id=? AND d.available_for_assignment=1", (deliverer_id,))
        current_assignments = cursor.fetchall()
        window['CurrentAssignments'].update(values=[f"{assignment[0]} - {assignment[1]}" for assignment in current_assignments])

    # Initial refresh of current assignments
    refresh_current_assignments()
    
    def create_deliverer_order_view_window(deliverer_id):
        # Fetch order details assigned to this deliverer, including receiver contact
        cursor.execute("""
            SELECT o.order_id, o.receiver_name, o.receiver_address, o.receiver_contact, o.gift_note, o.delivery_status, o.expected_delivery_date,
                   GROUP_CONCAT(f.f_name || ' (' || d.quantity || ')', ', ') AS flowers
            FROM Orders o
            JOIN OrderDetails d ON o.order_id = d.order_id
            JOIN Floral_Arrangement f ON d.flower_id = f.flower_ID
            WHERE o.deliverer_id=?
            GROUP BY o.order_id
        """, (deliverer_id,))
        assigned_orders = cursor.fetchall()

        layout = [
            [sg.Text("Your Assigned Orders")],
            [sg.Listbox(values=[
                f"Order ID: {order[0]} - Receiver: {order[1]} - Contact: {order[3]} - Address: {order[2]} - Gift Note: {order[4]} - Status: {order[5]} - Delivery Date: {order[6] or 'N/A'} - Flowers: {order[7]}"
                for order in assigned_orders], size=(100, 10), key='OrderList')],
            [sg.Button("Mark as Delivered"), sg.Button("Refresh"), sg.Button("Exit")]
        ]
        window = sg.Window("Assigned Orders", layout, finalize=True)

        while True:
            event, values = window.read()
            if event == 'Exit' or event == sg.WIN_CLOSED:
                break
            elif event == 'Refresh':
                # Re-fetch the order details
                cursor.execute("""
                    SELECT o.order_id, o.receiver_name, o.receiver_address, o.receiver_contact, o.gift_note, o.delivery_status, o.expected_delivery_date,
                           GROUP_CONCAT(f.f_name || ' (' || d.quantity || ')', ', ') AS flowers
                    FROM Orders o
                    JOIN OrderDetails d ON o.order_id = d.order_id
                    JOIN Floral_Arrangement f ON d.flower_id = f.flower_ID
                    WHERE o.deliverer_id=?
                    GROUP BY o.order_id
                """, (deliverer_id,))
                assigned_orders = cursor.fetchall()
                window['OrderList'].update(values=[
                    f"Order ID: {order[0]} - Receiver: {order[1]} - Contact: {order[3]} - Address: {order[2]} - Gift Note: {order[4]} - Status: {order[5]} - Delivery Date: {order[6] or 'N/A'} - Flowers: {order[7]}"
                    for order in assigned_orders])
            elif event == 'Mark as Delivered':
                if values['OrderList']:
                    selected_order = values['OrderList'][0]
                    order_id = selected_order.split(' - ')[0].split(': ')[1]
                    # Update the order as delivered
                    cursor.execute("UPDATE Orders SET delivery_status='Delivered', expected_delivery_date=CURRENT_DATE WHERE order_id=?", (order_id,))
                    conn.commit()
                    sg.popup("Order marked as delivered!")
                    # Refresh list
                    cursor.execute("""
                        SELECT o.order_id, o.receiver_name, o.receiver_address, o.receiver_contact, o.gift_note, o.delivery_status, o.expected_delivery_date,
                               GROUP_CONCAT(f.f_name || ' (' || d.quantity || ')', ', ') AS flowers
                        FROM Orders o
                        JOIN OrderDetails d ON o.order_id = d.order_id
                        JOIN Floral_Arrangement f ON d.flower_id = f.flower_ID
                        WHERE o.deliverer_id=?
                        GROUP BY o.order_id
                    """, (deliverer_id,))
                    assigned_orders = cursor.fetchall()
                    window['OrderList'].update(values=[
                        f"Order ID: {order[0]} - Receiver: {order[1]} - Contact: {order[3]} - Address: {order[2]} - Gift Note: {order[4]} - Status: {order[5]} - Delivery Date: {order[6] or 'N/A'} - Flowers: {order[7]}"
                        for order in assigned_orders])
                else:
                    sg.popup("Please select an order to mark as delivered.")

        window.close()




    

    while True:
        event, values = window.read()
        if event == 'Add Selected':
            selected_flowers = values['FlowerList']
            for flower in selected_flowers:
                flower_id = flower.split(' - ')[0]
                # Insert if not already assigned
                if not cursor.execute("SELECT 1 FROM DelivererAssignments WHERE deliverer_id=? AND flower_id=?", (deliverer_id, flower_id)).fetchone():
                    cursor.execute("INSERT INTO DelivererAssignments (deliverer_id, flower_id) VALUES (?, ?)", (deliverer_id, flower_id))
            conn.commit()
            refresh_current_assignments()  # Refresh the list of current assignments
            sg.popup("Selections updated!")
        elif event == 'Delete Selected':
            selected_assignments = values['CurrentAssignments']
            for assignment in selected_assignments:
                flower_id = assignment.split(' - ')[0]
                cursor.execute("DELETE FROM DelivererAssignments WHERE deliverer_id=? AND flower_id=?", (deliverer_id, flower_id))
            conn.commit()
            refresh_current_assignments()  # Refresh the list of current assignments
            sg.popup("Assignments updated!")
        elif event == 'View Assigned Orders':
            window.hide()
            create_deliverer_order_view_window(deliverer_id)
            window.un_hide()
        elif event in (sg.WIN_CLOSED, 'Exit'):
            break
    window.close()





def create_admin_assignment_window():
    cursor.execute("""
        SELECT a.deliverer_id, f.f_name, a.available_for_assignment
        FROM DelivererAssignments a
        JOIN Floral_Arrangement f ON a.flower_id = f.flower_ID
        WHERE a.available_for_assignment=1
    """)
    assignments = cursor.fetchall()
    layout = [
        [sg.Text("Deliverer Assignments")],
        [sg.Table(
            values=[[assignment[0], assignment[1], 'Yes' if assignment[2] else 'No'] for assignment in assignments],
            headings=['Deliverer ID', 'Flower Name', 'Available'],
            display_row_numbers=True,
            auto_size_columns=True,
            num_rows=min(10, len(assignments)),
            key='AssignmentsTable',
            select_mode=sg.TABLE_SELECT_MODE_BROWSE
        )],
        [sg.Button("Assign"), sg.Button("Close")]
    ]
    window = sg.Window("Admin Deliverer Assignments", layout)
    while True:
        event, values = window.read()
        if event == 'Assign':
            # Implementation for assigning specific orders to selected deliverers
            selected_row = values['AssignmentsTable'][0]
            selected_deliverer = assignments[selected_row][0]
            sg.popup(f"Order assigned to {selected_deliverer}!")
        elif event in (sg.WIN_CLOSED, 'Close'):
            break
    window.close()

def refresh_current_assignments(window, deliverer_id):
    cursor.execute("SELECT flower_id, f_name FROM DelivererAssignments JOIN Floral_Arrangement ON flower_id = flower_ID WHERE deliverer_id=? AND available_for_assignment=1", (deliverer_id,))
    current_assignments = cursor.fetchall()
    window['CurrentAssignments'].update(values=[f"{assignment[0]} - {assignment[1]}" for assignment in current_assignments])

def create_admin_order_assignment_window():
    # Fetch unassigned orders that need a deliverer, aggregating flower IDs
    cursor.execute("""
        SELECT o.order_id, o.customer_id, o.total_price, o.order_date, GROUP_CONCAT(d.flower_id, ', ') AS flower_ids
        FROM Orders o 
        JOIN OrderDetails d ON o.order_id = d.order_id
        WHERE o.deliverer_id IS NULL
        GROUP BY o.order_id
    """)
    unassigned_orders = cursor.fetchall()

    layout = [
        [sg.Text("Unassigned Orders")],
        [sg.Listbox(values=[f"Order ID: {order[0]} - Customer: {order[1]} - Total: ${order[2]:.2f} - Date: {order[3]} - Flower IDs: {order[4]}" for order in unassigned_orders], size=(80, 10), key='OrderList')],
        [sg.Text("Select a Deliverer for the Order:")],
        [sg.Combo(values=[], size=(30, 1), key='DelivererCombo')],
        [sg.Button("Assign Deliverer"), sg.Button("Exit")]
    ]
    window = sg.Window("Admin - Assign Orders to Deliverers", layout, finalize=True)

    while True:
        event, values = window.read()
        if event == 'Exit' or event == sg.WIN_CLOSED:
            break
        elif event == 'OrderList':
            if values['OrderList']:
                selected_order_details = values['OrderList'][0]
                flower_ids = set(selected_order_details.split(' - ')[4].split(': ')[1].split(', '))
                cursor.execute("SELECT deliverer_id FROM DelivererAssignments WHERE available_for_assignment=1")
                potential_deliverers = cursor.fetchall()

                valid_deliverers = []
                for deliverer in potential_deliverers:
                    cursor.execute("""
                        SELECT flower_id FROM DelivererAssignments
                        WHERE deliverer_id=? AND available_for_assignment=1
                    """, (deliverer[0],))
                    deliverer_flower_ids = set(flower[0] for flower in cursor.fetchall())
                    if flower_ids.issubset(deliverer_flower_ids):
                        valid_deliverers.append(deliverer[0])

                if valid_deliverers:
                    window['DelivererCombo'].update(values=valid_deliverers, disabled=False)
                else:
                    window['DelivererCombo'].update(values=[], disabled=True)  # Disable if no deliverers available
        elif event == 'Assign Deliverer':
            if values['OrderList'] and values['DelivererCombo']:
                selected_order_id = values['OrderList'][0].split(' - ')[0].split(': ')[1]  # Extract Order ID
                selected_deliverer = values['DelivererCombo']
                cursor.execute("UPDATE Orders SET deliverer_id=? WHERE order_id=?", (selected_deliverer, selected_order_id))
                conn.commit()
                sg.popup("Deliverer assigned successfully!")
                # Refresh the orders list
                cursor.execute("""
                    SELECT o.order_id, o.customer_id, o.total_price, o.order_date, GROUP_CONCAT(d.flower_id, ', ') AS flower_ids
                    FROM Orders o 
                    JOIN OrderDetails d ON o.order_id = d.order_id
                    WHERE o.deliverer_id IS NULL
                    GROUP BY o.order_id
                """)
                unassigned_orders = cursor.fetchall()
                window['OrderList'].update(values=[f"Order ID: {order[0]} - Customer: {order[1]} - Total: ${order[2]:.2f} - Date: {order[3]} - Flower IDs: {order[4]}" for order in unassigned_orders])
    window.close()


def create_order_selection_window():
    cursor.execute("""
        SELECT o.order_id, GROUP_CONCAT(d.flower_id, ', ') AS flower_ids
        FROM Orders o 
        JOIN OrderDetails d ON o.order_id = d.order_id
        WHERE o.deliverer_id IS NULL
        GROUP BY o.order_id
    """)
    unassigned_orders = cursor.fetchall()

    layout = [
        [sg.Text("Select an Order to Assign")],
        [sg.Listbox(values=[f"Order ID: {order[0]} - Flower IDs: {order[1]}" for order in unassigned_orders], size=(70, 10), key='OrderList')],
        [sg.Button("Next"), sg.Button("Exit")]
    ]
    window = sg.Window("Select Order", layout, finalize=True)

    while True:
        event, values = window.read()
        if event == 'Exit' or event == sg.WIN_CLOSED:
            window.close()
            break
        elif event == 'Next':
            if values['OrderList']:
                selected_order_details = values['OrderList'][0]
                window.close()
                create_deliverer_assignment_window(selected_order_details)
            else:
                sg.popup("Please select an order to proceed.")
    window.close()



def create_deliverer_assignment_window(selected_order_details):
    order_id, flower_ids = parse_order_details(selected_order_details)
    flower_ids = set(flower_ids.split(', '))  # Assuming flower_ids are a comma-separated string

    # Fetch deliverers who are capable of handling all specified flower IDs
    valid_deliverers = []
    for deliverer in fetch_all_deliverers():
        # Check if this deliverer can handle all required flower IDs
        if is_capable_deliverer(deliverer['id'], flower_ids):
            valid_deliverers.append(deliverer['id'])

    layout = [
        [sg.Text("Available Deliverers for Order ID: " + order_id)],
        [sg.Listbox(values=valid_deliverers, size=(30, 10), key='DelivererList')],
        [sg.Button("Assign"), sg.Button("Cancel")]
    ]
    window = sg.Window("Assign Deliverers", layout, finalize=True)

    while True:
        event, values = window.read()
        if event == 'Assign' and values['DelivererList']:
            deliverer_id = values['DelivererList'][0]
            assign_order_to_deliverer(order_id, deliverer_id)
            sg.popup("Order assigned successfully!")
            window.close()
        elif event in ("Cancel", sg.WIN_CLOSED):
            break
    window.close()

def parse_order_details(order_details):
    parts = order_details.split(' - ')
    order_id = parts[0].split(': ')[1]
    flower_ids = parts[1].split(': ')[1]
    return order_id, flower_ids

def fetch_all_deliverers():
    cursor.execute("SELECT DID FROM Deliverers")
    return [{'id': row[0]} for row in cursor.fetchall()]

def is_capable_deliverer(deliverer_id, required_flowers):
    cursor.execute("SELECT flower_id FROM DelivererAssignments WHERE deliverer_id=?", (deliverer_id,))
    deliverer_flowers = {row[0] for row in cursor.fetchall()}
    return required_flowers.issubset(deliverer_flowers)

def assign_order_to_deliverer(order_id, deliverer_id):
    cursor.execute("UPDATE Orders SET deliverer_id=? WHERE order_id=?", (deliverer_id, order_id))
    conn.commit()


login_window = sg.Window("Login", login_layout)

while True:
    event, values = login_window.read()
    
    if event == sg.WIN_CLOSED or event == 'Cancel':
        break
    elif event == 'Login':
        user_id = values['ID']
        password = values['Password']
        
        if cursor.execute("SELECT ID FROM Users WHERE ID=? AND password=?", (user_id, password)).fetchone():
            if cursor.execute("SELECT AID FROM Admins WHERE AID=?", (user_id,)).fetchone():
                login_window.hide()
                admin_window = create_admin_window()
                
                while True:
                    event, values = admin_window.read()
                    
                    if event in (sg.WIN_CLOSED, 'Logout'):
                        admin_window.close()
                        login_window.un_hide()
                        break
                    elif event == 'Add Flower Arrangement':
                        admin_window.hide()
                        flower_window = create_flower_window("Add")
                        while True:
                            event, values = flower_window.read()
                            if event == 'Submit':
                                if any(not values[field].strip() for field in ['flower_ID', 'f_size', 'f_type', 'quantity_in_stock', 'f_price', 'f_name', 'f_design']):
                                    sg.popup_error('All fields are required. Please fill out all fields.')
                                elif not values['flower_ID'].isnumeric():
                                    sg.popup('flower ıd should be numeric')
                                elif not values['quantity_in_stock'].isnumeric():
                                    sg.popup('fquantity should be numeric') 
                                elif not values['f_price'].isnumeric():
                                    sg.popup('fprice should be numeric')
                                else:
                                    try:
                                        cursor.execute('''
                                            INSERT INTO Floral_Arrangement (flower_ID, f_size, f_type, quantity_in_stock, f_price, f_name, f_design)
                                            VALUES (?, ?, ?, ?, ?, ?, ?)''', 
                                            (values['flower_ID'], values['f_size'], values['f_type'], values['quantity_in_stock'], 
                                             values['f_price'], values['f_name'], values['f_design']))
                                        conn.commit()
                                        sg.popup('New flower arrangement added!')
                                    except sqlite3.IntegrityError:
                                        sg.popup_error('Error: Flower ID already exists. Please enter a unique ID.')
                                flower_window.close()
                                break
                            elif event in (sg.WIN_CLOSED, 'Cancel'):
                                flower_window.close()
                                break
                        admin_window.un_hide()


                    elif event == 'Manage Floral Arrangements':
                        admin_window.hide()
                        manage_flowers_window = create_manage_flowers_window()
                        while True:
                            event, values = manage_flowers_window.read()
                            if event == 'Back':
                                manage_flowers_window.close()
                                admin_window.un_hide()
                                break
                            elif event == 'View':
                                if values['FlowerList']:
                                    manage_flower_details(values['FlowerList'][0], "View")
                                else:
                                    sg.popup_error('Please select a flower to view.', title='Selection Required')
                            elif event == 'Edit':
                                if values['FlowerList']:
                                    manage_flower_details(values['FlowerList'][0], "Edit")
                                else:
                                    sg.popup_error('Please select a flower to edit.', title='Selection Required')
                            elif event == 'Delete':
                                if values['FlowerList']:
                                    delete_flower(values['FlowerList'][0])
                                    manage_flowers_window.close()
                                    admin_window.un_hide()
                                    break
                                else:
                                    sg.popup_error('Please select a flower to delete.', title='Selection Required')
                    
                    elif event == 'Define Discounts':
                        discount_window = create_discount_window()
                        handle_discount_submission(discount_window)

                    elif event == 'View Discounts':
                        while True:
                            if create_view_discounts_window():  # Reopen if a deletion occurred
                                continue
                            break
                    elif event == 'Assign Orders to Deliverers':
                        admin_window.hide()
                        create_order_selection_window()
                        admin_window.un_hide()
                    elif event == sg.WIN_CLOSED or event == 'Logout':
                        break
                                    
            elif cursor.execute("SELECT CID FROM Customers WHERE CID=?", (user_id,)).fetchone():
                customer_id = user_id
                login_window.hide()
                customer_window = create_customer_window()
                cart_items=[]
                while True:
                    event, values = customer_window.read()
                    if event == 'Exit' or event == sg.WIN_CLOSED:
                        break
                    elif event == 'View Details':
                        if values['FlowerList']:
                            selected = values['FlowerList'][0].split(' - ')[0]
                            show_flower_details(selected)
                        else:
                            sg.popup_error('Please select a flower to view its details.')
                    elif event == 'Add to Cart':
                        if not values['FlowerList']:
                            sg.popup_error('Please select at least one flower to add to your cart.')
                        else:
                            add_to_cart(values['FlowerList'], cart_items)
                        sg.popup('Flower added')
                    elif event == 'View Cart':
                        customer_window.hide()
                        cart_window = create_cart_window(cart_items)
                        while True:
                            cart_event, cart_values = cart_window.read()
                            if cart_event in ['Continue Shopping', sg.WIN_CLOSED]:
                                cart_window.close()
                                customer_window.un_hide()
                                break
                            elif cart_event == 'Remove Selected':
                                if cart_values['CartList']:
                                    selected_item = cart_values['CartList'][0]
                                    remove_from_cart(selected_item, cart_items)
                                    cart_window['CartList'].update(values=[f"{item[0]} - {item[1]} - Qty: {item[3]}" for item in cart_items])
                            elif cart_event == 'Checkout':
                                if not cart_items:
                                    sg.popup_error("Your cart is empty. Please add items before checking out.")
                                else:
                                    gift_note_window = create_gift_note_window()
                                    while True:
                                        gift_event, gift_values = gift_note_window.read()
                                        if gift_event == "Submit Note":
                                            gift_note = gift_values['GiftNote'].strip()
                                            if gift_note:
                                                sg.popup(f"Thank you! Your gift note has been recorded:\n{gift_note}")
                                                gift_note_window.close()

                                                # Display the discounts and allow selection
                                                selected_discount = display_discounts_and_select()

                                                # Calculate the total price and apply discount if selected
                                                total_price = sum(float(item[2]) * int(item[3]) for item in cart_items)
                                                if selected_discount:
                                                    discount_percentage = int(selected_discount[3])
                                                    total_price *= (100 - discount_percentage) / 100
                                                    delete_used_discount(selected_discount[0])  # Delete the discount code after use

                                                # Collect receiver information
                                                receiver_info_window = create_receiver_info_window()
                                                receiver_event, receiver_values = receiver_info_window.read()
                                                if receiver_event == "Submit":
                                                    receiver_name = receiver_values['ReceiverName']
                                                    receiver_address = receiver_values['ReceiverAddress']
                                                    receiver_contact = receiver_values['ReceiverContact']
                                                    receiver_info_window.close()

                                                    # Generate a random order ID
                                                    order_id = generate_order_id()

                                                    # Insert order details into the database
                                                    # Insert order details into the database, including the default delivery status
                                                    insert_order_sql = '''
                                                    INSERT INTO Orders (order_id, customer_id, receiver_name, receiver_address, receiver_contact, total_price, delivery_status, gift_note)
                                                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                                                    '''
                                                    cursor.execute(insert_order_sql, (order_id, user_id, receiver_name, receiver_address, receiver_contact, total_price, "incomplete", gift_note))
                                                    conn.commit()
                                                    
                                                    for item in cart_items:
                                                        cursor.execute('''
                                                            INSERT INTO OrderDetails (order_id, flower_id, quantity)
                                                            VALUES (?, ?, ?)
                                                        ''', (order_id, item[0], item[3]))
                                                    conn.commit()


                                                    # Show order summary and confirmation
                                                    sg.popup(f"Order ID: {order_id}\nReceiver: {receiver_name}\nAddress: {receiver_address}\nContact: {receiver_contact}\nTotal Price: ${total_price:.2f}\nGift Note: {gift_note}",
                                                             title="Order Confirmation")
                                                    
                                                    cart_items.clear()  # Reset the cart
                                                    cart_window.close()
                                                    break

                                                elif receiver_event in ("Cancel", sg.WIN_CLOSED):
                                                    sg.popup("Order cancelled.")
                                                    receiver_info_window.close()
                                                break
                                            else:
                                                sg.popup_error("Please enter a note or cancel.")
                                        elif gift_event in ("Cancel", sg.WIN_CLOSED):
                                            gift_note_window.close()
                                            break
                    elif event == 'View Order History':
                        customer_window.hide()
                        create_customer_order_history_window(customer_id)
                        customer_window.un_hide()
                        


                                
                customer_window.close()
                login_window.un_hide()
            elif cursor.execute("SELECT DID FROM Deliverers WHERE DID=?", (user_id,)).fetchone():
                login_window.hide()
                create_deliverer_window(user_id)  # Show deliverer window
                login_window.un_hide()  # Show login window again after closing deliverer window
            else:
                sg.popup("Logged in, but no specific role found.")
        else:
            sg.popup("Incorrect ID or Password. Please try again.")

login_window.close()
